<a href="https://colab.research.google.com/github/erendagasan/Eren-Dagasan-Personal/blob/main/ARIMA-14.09.2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pmdarima

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from pmdarima.arima import auto_arima
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
import yfinance as yf

stock = "THYAO.IS"

dateparse = lambda dates: pd.datetime.strptime(dates, '%Y-%m-%d')
stock_data = yf.download(stock, start="2006-01-01", end="2023-09-14")

df_close = stock_data['Close']
df_log = np.log(df_close)
train_data, test_data = df_log[3:int(len(df_log)*0.9)], df_log[int(len(df_log)*0.9):]

model_autoARIMA = auto_arima(train_data, start_p=0, start_q=0,
                      test='adf',
                      max_p=3, max_q=3,
                      m=1,
                      d=None,
                      seasonal=False,
                      start_P=0,
                      D=0,
                      trace=True,
                      error_action='ignore',
                      suppress_warnings=True,
                      stepwise=True)

best_order = model_autoARIMA.order
print("Best ARIMA Order:", best_order)

[*********************100%%**********************]  1 of 1 completed
Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-18584.551, Time=0.75 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-18584.443, Time=0.49 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-18584.373, Time=1.11 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-18583.462, Time=0.27 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-18585.960, Time=4.08 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=-18583.871, Time=1.98 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=-18570.719, Time=5.25 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=-18583.755, Time=3.62 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=-18583.898, Time=1.16 sec


In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from datetime import datetime, timedelta

start="2023-08-28"
end="2023-09-11"

stock_data = yf.download(stock, start=start, end=end, progress=False)
new_date_index = stock_data.index + pd.Timedelta(days=(pd.Timestamp(end)-pd.Timestamp(start)+pd.Timedelta(1)).days)
model = ARIMA(np.log(stock_data["Close"]), order=best_order)
fitted = model.fit()

print(len(stock_data))

In [ ]:
forecast_results = fitted.get_forecast(steps=len(stock_data), alpha=0.05)

fc = forecast_results.predicted_mean
conf = forecast_results.conf_int()

fc_series = pd.Series(fc, index=stock_data.index)
lower_series = pd.DataFrame([new_date_index,conf.iloc[:, 0]]).T
# upper_series = pd.Series(stock_data.index, conf.iloc[:, 1])

plt.figure(figsize=(16,6), dpi=100)
# plt.plot(np.exp(train_data[3500:]), label='Training data', color="black")
plt.plot(np.exp(test_data[100:]), color='black', label='Test data')

# plt.plot(test_data.index, np.exp(fc), color='orange', label='Predicted Stock Price')  # Convert back to original scale
# plt.fill_between(test_data.index, np.exp(conf.iloc[:, 0]), np.exp(conf.iloc[:, 1]),
#                  color='k', alpha=.10)


from datetime import datetime, timedelta
today = datetime.now()
tomorrow = today + timedelta(days=1)
formatted_date = tomorrow.strftime("%Y-%m-%d")
today = today.strftime("%Y-%m-%d")

stock_data = yf.download(stock, start=end, end=formatted_date, progress=False)
plt.plot(stock_data.index, stock_data["Close"], color='red', label='Excluded stock data')

plt.plot(new_date_index, np.exp(fc), color='blue', label='Predicted stock price')  # Convert back to original scale
plt.fill_between(new_date_index, np.exp(conf.iloc[:, 0]), np.exp(conf.iloc[:, 1]),
                 color='k', alpha=.10)

plt.axhline(np.exp(conf.iloc[:, 1].max()), color="green")
plt.axhline(np.exp(conf.iloc[:, 0].min()), color="red")

print("FİYAT: ", round(stock_data["Close"].iloc[-1], 2), "TL\n")

print("ÜST PROJEKSİYON: ", round(np.exp(conf.iloc[:, 1].max()), 2), "TL")
print("ALT PROJEKSİYON: ", round(np.exp(conf.iloc[:, 0].min()), 2), "TL\n")

print("Günlük AL noktası: ", round(np.exp(float(lower_series[1][lower_series[0] == pd.Timestamp(today)])), 2), "TL")

plt.xlabel('Time')
plt.ylabel('Price')

plt.legend(loc='lower left', fontsize=8)
plt.show()

In [ ]:
model = ARIMA(train_data, order=best_order)
fitted = model.fit()

forecast_results = fitted.get_forecast(steps=len(test_data), alpha=0.05)

fc = forecast_results.predicted_mean
conf = forecast_results.conf_int()

fc_series = pd.Series(fc, index=test_data.index)
lower_series = pd.Series(conf.iloc[:, 0], index=test_data.index)
upper_series = pd.Series(conf.iloc[:, 1], index=test_data.index)

plt.figure(figsize=(16,6), dpi=100)
plt.plot(np.exp(train_data[1000:]), label='Training data', color="black")
plt.plot(np.exp(test_data), color='blue', label='Test data')

plt.plot(test_data.index, np.exp(fc), color='orange', label='Predicted Stock Price')  # Convert back to original scale
plt.fill_between(test_data.index, np.exp(conf.iloc[:, 0]), np.exp(conf.iloc[:, 1]),
                 color='k', alpha=.10)


plt.axhline(np.exp(conf.iloc[:, 1].max()), color="green")
plt.axhline(np.exp(conf.iloc[:, 0].min()), color="red")

plt.xlabel('Time')
plt.ylabel('Price')

plt.legend(loc='lower left', fontsize=8)
plt.show()

In [ ]:
# report performance
mse = mean_squared_error(test_data, fc)
print('MSE: '+str(mse))
mae = mean_absolute_error(test_data, fc)
print('MAE: '+str(mae))
rmse = math.sqrt(mean_squared_error(test_data, fc))
print('RMSE: '+str(rmse))
mape = np.mean(np.abs(fc.to_numpy() - test_data)/np.abs(test_data))
print('MAPE: '+str(mape))

MSE: 0.2674679937215887
MAE: 0.4483644851402625
RMSE: 0.5171730790766169
MAPE: 0.08581938413187133


In [ ]:
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from pmdarima.arima import auto_arima
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
import yfinance as yf

market = []

url = f"https://tr.tradingview.com/symbols/BIST-XU100/components/"
response = requests.get(url)

if response.status_code == 200:
  soup = BeautifulSoup(response.text, 'html.parser')
  elements = soup.find_all(class_="apply-common-tooltip tickerNameBox-GrtoTeat tickerName-GrtoTeat")

  for element in elements:
    text = element.get_text()
    market.append(text)

# nasdaq100 = ['MARA', 'AAPL', 'MSFT', 'GOOGL', 'GOOG', 'AMZN',
#              'NVDA', 'TSLA', 'META', 'AVGO', 'ASML',
#              'PEP', 'COST', 'ADBE', 'AZN', 'CSCO',
#              'NFLX', 'AMD', 'CMCSA', 'TMUS', 'TXN',
#              'QCOM', 'HON', 'INTU', 'INTC', 'SNY',
#              'VZ', 'AMGN', 'SBUX', 'ISRG', 'AMAT',
#              'BKNG', 'ADI', 'MDLZ', 'PDD', 'GILD',
#              'ADP', 'VRTX', 'ABNB', 'LRCX', 'PYPL',
#              'REGN', 'EQIX', 'MU', 'CSX', 'SNPS',
#              'CME', 'CDNS', 'KLAC', 'NTES']

sheet_id = "1RSqOXkFTAO7g4H9LEY3d3IX6H6bJaYk1"
sheet_name = "Sheet_1"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
result_df = pd.read_csv(url)

bist_best_orders = pd.DataFrame()

for stock in result_df["STOCK"].unique():
  try:
    print(stock)
    stock = stock+".IS"
    dateparse = lambda dates: pd.datetime.strptime(dates, '%Y-%m-%d')
    stock_data = yf.download(stock, start="2006-01-01", end="2023-09-14", progress=False)

    df_close = stock_data['Close']
    df_log = np.log(df_close)
    train_data, test_data = df_log[3:int(len(df_log)*0.9)], df_log[int(len(df_log)*0.9):]

    model_autoARIMA = auto_arima(train_data, start_p=0, start_q=0,
                          test='adf',
                          max_p=3, max_q=3,
                          m=1,
                          d=None,
                          seasonal=False,
                          start_P=0,
                          D=0,
                          trace=False,
                          error_action='ignore',
                          suppress_warnings=True,
                          stepwise=True)

    best_order = model_autoARIMA.order

    bist_best_orders = pd.concat([bist_best_orders, pd.DataFrame([[stock, best_order]], columns=["STOCK", "BEST ORDER"])])

    print("Best param search is done. \n")
    # print("Best ARIMA Order:", best_order)

    start="2023-08-28"
    end="2023-09-11"

    stock_data = yf.download(stock, start=start, end=end, progress=False)
    new_date_index = stock_data.index + pd.Timedelta(days=(pd.Timestamp(end)-pd.Timestamp(start)+pd.Timedelta(1)).days)
    model = ARIMA(np.log(stock_data["Close"]), order=best_order)
    fitted = model.fit()

    # print(len(stock_data))

    forecast_results = fitted.get_forecast(steps=len(stock_data), alpha=0.05)

    fc = forecast_results.predicted_mean
    conf = forecast_results.conf_int()

    fc_series = pd.Series(fc, index=stock_data.index)
    lower_series = pd.DataFrame([new_date_index, conf.iloc[:, 0]]).T

    from datetime import datetime, timedelta
    today = datetime.now()
    tomorrow = today + timedelta(days=1)
    formatted_date = tomorrow.strftime("%Y-%m-%d")
    today = today.strftime("%Y-%m-%d")

    stock_data = yf.download(stock, start="2023-09-01", end="2023-09-15", progress=False)

    # if (round(stock_data["Close"].iloc[-1], 2) < round(np.exp(float(lower_series[1][lower_series[0] == pd.Timestamp(today)])), 2)) or (round(np.exp(conf.iloc[:, 1].max()), 2) > 2*(round(np.exp(conf.iloc[:, 0].min()), 2))) or (np.exp(fc.max() > round(stock_data["Close"].iloc[-1], 2))):

    if (round(stock_data["Close"].iloc[-1], 2) < round(np.exp(float(lower_series[1][lower_series[0] == pd.Timestamp(today)])), 2)) or (round(np.exp(conf.iloc[:, 1].max()), 2) > 3*(round(np.exp(conf.iloc[:, 0].min()), 2))) or (np.exp(fc.max()) > 1.2*round(stock_data["Close"].iloc[-1], 2)):
      print("****************",stock,"****************", "\n")
  except:
    pass

In [28]:
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from pmdarima.arima import auto_arima
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
import yfinance as yf

market = []

url = f"https://tr.tradingview.com/symbols/BIST-XU100/components/"
response = requests.get(url)

if response.status_code == 200:
  soup = BeautifulSoup(response.text, 'html.parser')
  elements = soup.find_all(class_="apply-common-tooltip tickerNameBox-GrtoTeat tickerName-GrtoTeat")

  for element in elements:
    text = element.get_text()
    market.append(text)

# nasdaq100 = ['MARA', 'AAPL', 'MSFT', 'GOOGL', 'GOOG', 'AMZN',
#              'NVDA', 'TSLA', 'META', 'AVGO', 'ASML',
#              'PEP', 'COST', 'ADBE', 'AZN', 'CSCO',
#              'NFLX', 'AMD', 'CMCSA', 'TMUS', 'TXN',
#              'QCOM', 'HON', 'INTU', 'INTC', 'SNY',
#              'VZ', 'AMGN', 'SBUX', 'ISRG', 'AMAT',
#              'BKNG', 'ADI', 'MDLZ', 'PDD', 'GILD',
#              'ADP', 'VRTX', 'ABNB', 'LRCX', 'PYPL',
#              'REGN', 'EQIX', 'MU', 'CSX', 'SNPS',
#              'CME', 'CDNS', 'KLAC', 'NTES']

bist_best_orders = pd.DataFrame()

for stock in market:
  try:
    print(stock)
    stock = stock+".IS"

    order_df = pd.read_csv("bist100_best_orders.csv")

    best_order = order_df[order_df["STOCK"] == stock]["BEST ORDER"]

    best_order = model_autoARIMA.order

    print("Best param retrieval is done. \n")
    # print("Best ARIMA Order:", best_order)

    start="2023-08-28"
    end="2023-09-11"

    stock_data = yf.download(stock, start=start, end=end, progress=False)
    new_date_index = stock_data.index + pd.Timedelta(days=(pd.Timestamp(end)-pd.Timestamp(start)+pd.Timedelta(1)).days)
    model = ARIMA(np.log(stock_data["Close"]), order=best_order)
    fitted = model.fit()

    # print(len(stock_data))

    forecast_results = fitted.get_forecast(steps=len(stock_data), alpha=0.05)

    fc = forecast_results.predicted_mean
    conf = forecast_results.conf_int()

    fc_series = pd.Series(fc, index=stock_data.index)
    lower_series = pd.DataFrame([new_date_index, conf.iloc[:, 0]]).T

    from datetime import datetime, timedelta
    today = datetime.now()
    tomorrow = today + timedelta(days=1)
    formatted_date = tomorrow.strftime("%Y-%m-%d")
    today = today.strftime("%Y-%m-%d")

    stock_data = yf.download(stock, start="2023-09-01", end="2023-09-15", progress=False)

    # if (round(stock_data["Close"].iloc[-1], 2) < round(np.exp(float(lower_series[1][lower_series[0] == pd.Timestamp(today)])), 2)) or (round(np.exp(conf.iloc[:, 1].max()), 2) > 2*(round(np.exp(conf.iloc[:, 0].min()), 2))) or (np.exp(fc.max() > round(stock_data["Close"].iloc[-1], 2))):

    if (round(stock_data["Close"].iloc[-1], 2) < round(np.exp(float(lower_series[1][lower_series[0] == pd.Timestamp(today)])), 2)) or (round(np.exp(conf.iloc[:, 1].max()), 2) > 3*(round(np.exp(conf.iloc[:, 0].min()), 2))) or (np.exp(fc.max()) > 1.2*round(stock_data["Close"].iloc[-1], 2)):
      print("****************",stock,"****************", "\n")
  except:
    pass

AEFES
Best param retrieval is done. 

AGHOL
Best param retrieval is done. 

AHGAZ
Best param retrieval is done. 

AKBNK
Best param retrieval is done. 

AKCNS
Best param retrieval is done. 

AKFGY
Best param retrieval is done. 

AKSA
Best param retrieval is done. 

AKSEN
Best param retrieval is done. 

ALARK
Best param retrieval is done. 

ALBRK
Best param retrieval is done. 



ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['ALFAS.IS']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")


**************** ALBRK.IS **************** 

ALFAS
Best param retrieval is done. 

ARCLK
Best param retrieval is done. 



ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['ARCLK.IS']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2023-08-28 -> 2023-09-11)')


ASELS
Best param retrieval is done. 

ASTOR
Best param retrieval is done. 



...

FROTO
Best param retrieval is done. 

GARAN
Best param retrieval is done. 

GENIL
Best param retrieval is done. 

GESAN
Best param retrieval is done. 

GLYHO
Best param retrieval is done. 

GSDHO
Best param retrieval is done. 

GUBRF
Best param retrieval is done. 

GWIND
Best param retrieval is done. 

HALKB
Best param retrieval is done. 

HEKTS
Best param retrieval is done. 

IPEKE
Best param retrieval is done. 

ISCTR
Best param retrieval is done. 

ISDMR
Best param retrieval is done. 

ISGYO
Best param retrieval is done. 

ISMEN
Best param retrieval is done. 

**************** ISMEN.IS **************** 

IZMDC
Best param retrieval is done. 

**************** IZMDC.IS **************** 

KARSN
Best param retrieval is done. 



ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['KCAER.IS']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")


KCAER
Best param retrieval is done. 

KCHOL
Best param retrieval is done. 

KMPUR
Best param retrieval is done. 

KONTR
Best param retrieval is done. 

KONYA
Best param retrieval is done. 

**************** KONYA.IS **************** 

KORDS
Best param retrieval is done. 

KOZAA
Best param retrieval is done. 

KOZAL
Best param retrieval is done. 

KRDMD
Best param retrieval is done. 

KZBGY
Best param retrieval is done. 

**************** KZBGY.IS **************** 

MAVI
Best param retrieval is done. 

MGROS
Best param retrieval is done. 

ODAS
Best param retrieval is done. 

**************** ODAS.IS **************** 

OTKAR
Best param retrieval is done. 

OYAKC
Best param retrieval is done. 

PENTA
Best param retrieval is done. 

PETKM
Best param retrieval is done. 

**************** PETKM.IS **************** 

PGSUS
Best param retrieval is done. 



ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['PSGYO.IS']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")


PSGYO
Best param retrieval is done. 

QUAGR
Best param retrieval is done. 

**************** QUAGR.IS **************** 

SAHOL
Best param retrieval is done. 

SASA
Best param retrieval is done. 

**************** SASA.IS **************** 

SELEC
Best param retrieval is done. 

SISE
Best param retrieval is done. 

SKBNK
Best param retrieval is done. 

SMRTG
Best param retrieval is done. 

SNGYO
Best param retrieval is done. 

SOKM
Best param retrieval is done. 

TAVHL
Best param retrieval is done. 

TCELL
Best param retrieval is done. 

THYAO
Best param retrieval is done. 

**************** THYAO.IS **************** 

TKFEN
Best param retrieval is done. 

TKNSA
Best param retrieval is done. 

TOASO
Best param retrieval is done. 

TSKB
Best param retrieval is done. 

TTKOM
Best param retrieval is done. 

**************** TTKOM.IS **************** 

TTRAK
Best param retrieval is done. 

TUKAS
Best param retrieval is done. 

TUPRS
Best param retrieval is done. 

ULKER
Best param retrieval 